# Santander Kaggle
## Customer Transaction Prediction


<img src="https://i.imgur.com/3jkg3yS.jpg" />





## Neste desafio será necessário prever a variável target do DataSet

## Santander: Overview da proposta do desafio

 No Santander, nossa missão é ajudar pessoas e empresas a prosperar. Estamos sempre procurando maneiras de ajudar nossos clientes a entender sua saúde financeira e identificar quais produtos e serviços podem ajudá-los a atingir suas metas monetárias.

 Nossa equipe de ciência de dados está desafiando continuamente nossos algoritmos de aprendizado de máquina, trabalhando com a comunidade global de dados científicos para garantir que possamos identificar com mais precisão novas maneiras de resolver nosso desafio mais comum, problemas de classificação binária como: um cliente está satisfeito? Um cliente comprará este produto? Um cliente pode pagar este empréstimo?

 Neste desafio, convidamos a Kagglers a nos ajudar a identificar quais clientes farão uma transação específica no futuro, independentemente do volume de dinheiro transacionado. Os dados fornecidos para esta competição têm a mesma estrutura que os dados reais que temos disponíveis para resolver este problema.

In [ ]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import folium
%matplotlib inline



import warnings
warnings.filterwarnings("ignore")

In [ ]:
%time train_data = pd.read_csv('../input/santander-customer-transaction-prediction/train.csv')
%time test_data = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

# Verificando a variável target:

In [ ]:
sns.countplot(y=train_data.target ,data=train_data)
plt.xlabel("Contagem dos valores da target")
plt.ylabel("Target")
plt.show()

Note que a um desbalenaceamento da target, mas vamos prosseguir mesmo com esse desbalanecamento.

**Vamos verificar se a desbalanceamento na demais variáveis:**

In [ ]:
train_data.hist(figsize=(30,24),bins = 15)
plt.title("Distribuição das variáveis")
plt.show()

Note que as variávies então próximas de uma curva normal ou seja não estão muito desbalanceadas, estão muito boas logo seguiremos para os próximos passo. 

# Redução de dimensionalidade 'PCA'

Por que o PCA agora ?

Resposta: O PCA é usado principalmente como uma ferramenta na análise exploratória de dados e na criação de modelos preditivos.

Se tivermos dados de alta dimensão, pode ser difícil plotá-lo de maneira eficaz. Às vezes, a plotagem dos dois primeiros componentes principais pode revelar uma estrutura geométrica interessante nos dados.

E ele encontra atributos / variáveis ​​ essenciais (seleção de recursos em dados de alta dimensão)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

mmscale = MinMaxScaler()  
X_train = mmscale.fit_transform(train_data.drop(['ID_code','target'],axis=1))  
X_test = mmscale.transform(test_data.drop(['ID_code'], axis=1))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()  
a = pca.fit_transform(X_train) 
b = pca.transform(X_test)

In [ ]:
variancia_pca = pca.explained_variance_ratio_

In [ ]:
pd.DataFrame(variancia_pca,columns=['Variância após o PCA']).plot(kind='box')

In [ ]:
with plt.style.context('classic'):
    plt.figure(figsize=(12, 9))

    plt.bar(range(200), variancia_pca, alpha=0.5, align='center',
            label='variância individual')
    plt.ylabel('Taxa de variância')
    plt.xlabel('Componentes principais')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:
sum(variancia_pca[:100])

**Conclusão: Não usaremos o PCA, pois encotramos baixa correlação entre as variáveis, portanto o PCA não ficaria eficaz nesse cenário:**

# Vamos criar um Modelo com Gaussian Naive

In [ ]:
X_train = train_data.iloc[:, 2:].values.astype('float64')
y_train = train_data['target'].values

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import QuantileTransformer

pipeline = make_pipeline(QuantileTransformer(output_distribution='normal'), GaussianNB())
pipeline.fit(X_train, y_train)

Após o treinamento do modelo, plotamos a curva ROC nos dados de treinamento e avaliamos o modelo calculando a AUC do treinamento e a AUC para validação

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thr = roc_curve(y_train, pipeline.predict_proba(X_train)[:,1])
plt.plot(fpr, tpr)
plt.xlabel('Taxa de falsos positivos')
plt.ylabel('Taxa verdadeiros positivos')
plt.title('Curva ROC')
auc(fpr, tpr)

Aplicando o Cross-Validation para obter um score:

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=10).mean()

**Conclução: Alcançamos uma boa AUC em treinamento e uma boa Cross-Validation com score de 0.889**